# Imports and Common Functions

In [89]:
import requests
import json
import pandas as pd
import numpy as np
import datetime

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Loading in data from Parse

In [2]:
# URL and header for Parse
base_url = 'https://dtr-les.herokuapp.com/parse/classes/'
header = {'X-Parse-Application-Id': 'PkngqKtJygU9WiQ1GXM9eC0a17tKmioKKmpWftYr'}
data = {'limit': '10000'}

# hotspots
resp = requests.get(base_url + 'hotspot', headers=header, data=data)
hotspots = pd.DataFrame(resp.json()['results'])

hotspot_string_cols = ['objectId', 'vendorId', 'archiver', 'locationCommonName', \
                       'tag', 'submissionMethod', 'beaconId']
hotspots['createdAt'] = (pd.to_datetime(hotspots['createdAt']).astype(np.int64) // 10**9)
hotspots['updatedAt'] = (pd.to_datetime(hotspots['updatedAt']).astype(np.int64) // 10**9)
hotspots[hotspot_string_cols] = hotspots[hotspot_string_cols].astype(str)

# sent notifications
resp = requests.get(base_url + 'notificationSent', headers=header, data=data)
notification_sent = pd.DataFrame(resp.json()['results'])

notification_sent_string_cols = ['objectId', 'vendorId', 'hotspotId', 'notificationString']
notification_sent['createdAt'] = (pd.to_datetime(notification_sent['createdAt']).astype(np.int64) // 10**9)
notification_sent['updatedAt'] = (pd.to_datetime(notification_sent['updatedAt']).astype(np.int64) // 10**9)
notification_sent[notification_sent_string_cols] = notification_sent[notification_sent_string_cols].astype(str)

# notification responses
resp = requests.get(base_url + 'pingResponse', headers=header, data=data)
notification_responses = pd.DataFrame(resp.json()['results'])

notification_responses_string_cols = ['objectId', 'hotspotId', 'vendorId', 'question', 'response', 'tag']
notification_responses['createdAt'] = (pd.to_datetime(notification_responses['createdAt']).astype(np.int64) // 10**9)
notification_responses['updatedAt'] = (pd.to_datetime(notification_responses['updatedAt']).astype(np.int64) // 10**9)
notification_responses[notification_responses_string_cols] = \
    notification_responses[notification_responses_string_cols].astype(str)

# users
resp = requests.get(base_url + 'user', headers=header, data=data)
users = pd.DataFrame(resp.json()['results'])

user_string_cols = ['objectId', 'firstName', 'lastName', 'vendorId', \
                    'firstPreference', 'secondPreference', 'thirdPreference', 'fourthPreference']
users['createdAt'] = (pd.to_datetime(users['createdAt']).astype(np.int64) // 10**9)
users['updatedAt'] = (pd.to_datetime(users['updatedAt']).astype(np.int64) // 10**9)
users[user_string_cols] = users[user_string_cols].astype(str)

# pretracking debug (to see app terminations) 
resp = requests.get(base_url + 'pretracking_debug', headers=header, data=data)
debug = pd.DataFrame(resp.json()['results'])

debug_string_cols = ['objectId', 'vendor_id', 'console_string']
debug['createdAt'] = (pd.to_datetime(debug['createdAt']).astype(np.int64) // 10**9)
debug['updatedAt'] = (pd.to_datetime(debug['updatedAt']).astype(np.int64) // 10**9)
debug[debug_string_cols] = debug[debug_string_cols].astype(str)

# Clean Data

In [3]:
# filter data for timestamps between CST start and end time
start_time = 1494219600 # 05-08-2017 00:00:00
end_time   = 1495342800 # 05-21-2017 00:00:00

# remove all of kapil's ids 
kapil_vendor_id = 'F1AE595D-F244-4367-8744-27CA60450F0D'

In [4]:
hotspots = hotspots[(hotspots['createdAt'] >= start_time) & (hotspots['createdAt'] < end_time) & \
                    (hotspots['vendorId'] != kapil_vendor_id)].reset_index(drop=True)

notification_sent = notification_sent[(notification_sent['createdAt'] >= start_time) & (notification_sent['createdAt'] < end_time) & \
                                      (notification_sent['vendorId'] != kapil_vendor_id)].reset_index(drop=True)

notification_responses = notification_responses[(notification_responses['createdAt'] >= start_time) & (notification_responses['createdAt'] < end_time) & \
                                                (notification_responses['vendorId'] != kapil_vendor_id)].reset_index(drop=True)

users = users[(users['createdAt'] >= start_time) & (users['createdAt'] < end_time) & \
              (users['vendorId'] != kapil_vendor_id)].reset_index(drop=True)

debug = debug[(debug['createdAt'] >= start_time) & (debug['createdAt'] < end_time) & \
              (debug['vendor_id'] != kapil_vendor_id)].reset_index(drop=True)

In [5]:
# delete unnecessary columns
del notification_sent['locationKey']
del notification_sent['notificationLevel']

# Analysis 

## Combine notification_sent and notification_responses to analyze response rate

In [6]:
# merge notification sent and notification responses
merged_notifications = notification_sent.merge(notification_responses[['hotspotId', 'vendorId', 'timestamp', 'question', 'response']], \
                                               on=['hotspotId','vendorId'], how='left')

# add in location data from hotspot df
merged_notifications = merged_notifications.merge(hotspots[['objectId', 'tag', 'locationCommonName']], \
                                                  left_on='hotspotId', right_on='objectId', how='left')

# delete extra columns and rename
del merged_notifications['objectId_y']
merged_notifications.rename(columns={'timestamp_x': 'timestamp_sent', 'timestamp_y': 'timestamp_response', 'objectId_x': 'objectId'}, \
                            inplace=True)

# replace NAs
merged_notifications['timestamp_response'].fillna(-1, inplace=True)
merged_notifications['tag'].fillna('', inplace=True)
merged_notifications['question'].fillna('', inplace=True)
merged_notifications['response'].fillna('missed notification', inplace=True)

# type cast timestamp_response
merged_notifications['timestamp_response'] = merged_notifications['timestamp_response'].astype(np.int64)

# keep only last if there are duplicate rows
merged_notifications['vendor_hotspot_id'] = merged_notifications['vendorId'] + '_' + merged_notifications['hotspotId']
merged_notifications = merged_notifications[~(merged_notifications.duplicated(subset='vendor_hotspot_id', keep='last'))]

# show data frame
merged_notifications

,createdAt,gmtOffset,hotspotId,notificationString,objectId,timestamp_sent,updatedAt,vendorId,timestamp_response,question,response,tag,locationCommonName,vendor_hotspot_id
2,1494266124,-18000,e3j2GqkyMA,"Notified for e3j2GqkyMA (42.056922, -87.676545...",zyHdCC8auP,1494266124,1494266124,FDFE420C-04ED-47D6-AADF-8909DD559659,1494266137,tablesavailable,yes,workspaces,Delta Lab/Hackerspace,FDFE420C-04ED-47D6-AADF-8909DD559659_e3j2GqkyMA
3,1494302095,-18000,GALr2LpTzN,"Notified for GALr2LpTzN (42.055781, -87.674974...",00QYqCPK2B,1494302094,1494302095,83FF9726-945C-4D9B-9475-A41DC2A41CF4,-1,,missed notification,cafes,Food for Thought at Garrett,83FF9726-945C-4D9B-9475-A41DC2A41CF4_GALr2LpTzN
5,1494302551,-18000,8xoMpll7UZ,"Notified for 8xoMpll7UZ (42.060275, -87.675714...",Dm1dUaAF4M,1494302488,1494302551,83FF9726-945C-4D9B-9475-A41DC2A41CF4,1494302551,linelength,short: < 5 people,cafes,Lisas Cafe,83FF9726-945C-4D9B-9475-A41DC2A41CF4_8xoMpll7UZ
6,1494303940,-18000,C4a4c3ME34,"Notified for C4a4c3ME34 (42.058332, -87.683737...",GGpmnZSS2R,1494303936,1494303940,FDFE420C-04ED-47D6-AADF-8909DD559659,-1,,missed notification,coffeeshops,Coffee Lab,FDFE420C-04ED-47D6-AADF-8909DD559659_C4a4c3ME34
7,1494340949,-18000,3ReQ6w8r0G,"Notified for 3ReQ6w8r0G (42.054274, -87.678216...",hG4qXuwlZ2,1494340949,1494340949,0920AC3D-8789-43D7-8FF7-D800C2836F39,-1,,missed notification,gyms,Blomquist,0920AC3D-8789-43D7-8FF7-D800C2836F39_3ReQ6w8r0G
20,1494345108,-18000,gANyyHNOFM,"Notified for gANyyHNOFM (42.056922, -87.676545...",mn2c1qn0Or,1494345107,1494345108,0920AC3D-8789-43D7-8FF7-D800C2836F39,-1,,missed notification,workspaces,Delta Lab/Hackerspace,0920AC3D-8789-43D7-8FF7-D800C2836F39_gANyyHNOFM
21,1494345287,-18000,gANyyHNOFM,"Notified for gANyyHNOFM (42.056922, -87.676545...",Cou0EdiU0i,1494345287,1494345287,ADDD9CDC-AFFE-4958-8AB4-70E8D28E45E1,-1,,missed notification,workspaces,Delta Lab/Hackerspace,ADDD9CDC-AFFE-4958-8AB4-70E8D28E45E1_gANyyHNOFM
36,1494348712,-18000,HBzYGSKU51,"Notified for HBzYGSKU51 (42.056922, -87.676545...",RCvGWwIoYo,1494348712,1494348712,0920AC3D-8789-43D7-8FF7-D800C2836F39,-1,,missed notification,workspaces,Delta Lab/Hackerspace,0920AC3D-8789-43D7-8FF7-D800C2836F39_HBzYGSKU51
37,1494349304,-18000,HBzYGSKU51,"Notified for HBzYGSKU51 (42.056922, -87.676545...",TIVjJfUEla,1494349304,1494349304,ADDD9CDC-AFFE-4958-8AB4-70E8D28E45E1,-1,,missed notification,workspaces,Delta Lab/Hackerspace,ADDD9CDC-AFFE-4958-8AB4-70E8D28E45E1_HBzYGSKU51
38,1494350035,-18000,Z4MyGxg6Gn,"Notified for Z4MyGxg6Gn (42.059284, -87.673826...",qAyIpozxQd,1494350035,1494350035,ADDD9CDC-AFFE-4958-8AB4-70E8D28E45E1,-1,,missed notification,gyms,SPAC,ADDD9CDC-AFFE-4958-8AB4-70E8D28E45E1_Z4MyGxg6Gn


### General Response Rate

In [7]:
# count notification responses by type
valid_notifications = merged_notifications[~merged_notifications['response'].isin(['missed notification', 'I don\'t know'])]
idk_notifications = merged_notifications[merged_notifications['response'].isin(['I don\'t know'])]
missed_notifications = merged_notifications[merged_notifications['response'].isin(['missed notification'])]
non_missed_notifications = merged_notifications[~merged_notifications['response'].isin(['missed notification'])]

valid_notification_count = len(valid_notifications)
idk_notifications_count = len(idk_notifications)
missed_notifications_count = len(missed_notifications)
non_missed_notifications_count = len(non_missed_notifications)
total_notifications = len(merged_notifications)

In [8]:
print('valid responses/total responses: {}/{} | Proportion: {}%'.format(valid_notification_count, total_notifications, round(100 * valid_notification_count / total_notifications, 2)))
print('idk responses/total responses: {}/{} | Proportion: {}%'.format(idk_notifications_count, total_notifications, round(100 * idk_notifications_count / total_notifications, 2)))
print('missed notifications/total responses: {}/{} | Proportion: {}%'.format(missed_notifications_count, total_notifications, round(100 * missed_notifications_count / total_notifications, 2)))

valid responses/total responses: 110/393 | Proportion: 27.99%
idk responses/total responses: 73/393 | Proportion: 18.58%
missed notifications/total responses: 210/393 | Proportion: 53.44%


In [9]:
print('valid responses/non-missed responses: {}/{} | Proportion: {}%'.format(valid_notification_count, non_missed_notifications_count, round(100 * valid_notification_count / non_missed_notifications_count, 2)))
print('idk responses/non-missed responses: {}/{} | Proportion: {}%'.format(idk_notifications_count, non_missed_notifications_count, round(100 * idk_notifications_count / non_missed_notifications_count, 2)))
print('non-missed (either valid response or idk) notifications/total responses: {}/{} | Proportion: {}%'.format(non_missed_notifications_count, total_notifications, round(100 * non_missed_notifications_count / total_notifications, 2)))

valid responses/non-missed responses: 110/183 | Proportion: 60.11%
idk responses/non-missed responses: 73/183 | Proportion: 39.89%
non-missed (either valid response or idk) notifications/total responses: 183/393 | Proportion: 46.56%


### Reponse rate by hotspot category

In [10]:
total_bytag = pd.DataFrame({'count': merged_notifications.groupby('tag')['objectId'].count()}).reset_index()
valid_bytag = pd.DataFrame({'count': valid_notifications.groupby('tag')['objectId'].count()}).reset_index()
idk_bytag = pd.DataFrame({'count': idk_notifications.groupby('tag')['objectId'].count()}).reset_index()
non_missed_bytag = pd.DataFrame({'count': non_missed_notifications.groupby('tag')['objectId'].count()}).reset_index()
missed_bytag = pd.DataFrame({'count': missed_notifications.groupby('tag')['objectId'].count()}).reset_index()

#### Over Total Notifications

In [11]:
total_valid_bytag = valid_bytag.merge(total_bytag, on='tag', how='outer')
total_valid_bytag.fillna(0, inplace=True)
total_valid_bytag.rename(columns={'count_x': 'count', 'count_y': 'total'}, inplace=True)
total_valid_bytag['proportion'] = round(100 * total_valid_bytag['count'] / total_valid_bytag['total'], 2)

total_idk_bytag = idk_bytag.merge(total_bytag, on='tag', how='outer')
total_idk_bytag.fillna(0, inplace=True)
total_idk_bytag.rename(columns={'count_x': 'count', 'count_y': 'total'}, inplace=True)
total_idk_bytag['proportion'] = round(100 * total_idk_bytag['count'] / total_idk_bytag['total'], 2)

total_missed_bytag = missed_bytag.merge(total_bytag, on='tag', how='outer')
total_missed_bytag.fillna(0, inplace=True)
total_missed_bytag.rename(columns={'count_x': 'count', 'count_y': 'total'}, inplace=True)
total_missed_bytag['proportion'] = round(100 * total_missed_bytag['count'] / total_missed_bytag['total'], 2)

In [12]:
print('Valid Responses by Category (Total Notifications): \n{}\n'.format(total_valid_bytag))
print('IDK Responses by Category(Total Notifications): \n{}\n'.format(total_idk_bytag))
print('Missed Notifications by Category(Total Notifications): \n{}\n'.format(total_missed_bytag))

Valid Responses by Category (Total Notifications): 
           tag  count  total  proportion
0        cafes     23     94       24.47
1  coffeeshops     38    156       24.36
2         food      1      1      100.00
3         gyms      9     47       19.15
4   workspaces     39     95       41.05

IDK Responses by Category(Total Notifications): 
           tag  count  total  proportion
0        cafes   18.0     94       19.15
1  coffeeshops   35.0    156       22.44
2         gyms    9.0     47       19.15
3   workspaces   11.0     95       11.58
4         food    0.0      1        0.00

Missed Notifications by Category(Total Notifications): 
           tag  count  total  proportion
0        cafes   53.0     94       56.38
1  coffeeshops   83.0    156       53.21
2         gyms   29.0     47       61.70
3   workspaces   45.0     95       47.37
4         food    0.0      1        0.00



#### Over Non-missed Notifications

In [13]:
non_missed_valid_bytag = valid_bytag.merge(non_missed_bytag, on='tag', how='outer')
non_missed_valid_bytag.fillna(0, inplace=True)
non_missed_valid_bytag.rename(columns={'count_x': 'count', 'count_y': 'total'}, inplace=True)
non_missed_valid_bytag['proportion'] = round(100 * non_missed_valid_bytag['count'] / non_missed_valid_bytag['total'], 2)

non_missed_idk_bytag = idk_bytag.merge(non_missed_bytag, on='tag', how='outer')
non_missed_idk_bytag.fillna(0, inplace=True)
non_missed_idk_bytag.rename(columns={'count_x': 'count', 'count_y': 'total'}, inplace=True)
non_missed_idk_bytag['proportion'] = round(100 * non_missed_idk_bytag['count'] / non_missed_idk_bytag['total'], 2)

In [14]:
print('Valid Responses by Category (Non-Missed Notification): \n{}\n'.format(non_missed_valid_bytag))
print('IDK Responses by Category (Non-Missed Notification): \n{}\n'.format(non_missed_idk_bytag))

Valid Responses by Category (Non-Missed Notification): 
           tag  count  total  proportion
0        cafes     23     41       56.10
1  coffeeshops     38     73       52.05
2         food      1      1      100.00
3         gyms      9     18       50.00
4   workspaces     39     50       78.00

IDK Responses by Category (Non-Missed Notification): 
           tag  count  total  proportion
0        cafes   18.0     41       43.90
1  coffeeshops   35.0     73       47.95
2         gyms    9.0     18       50.00
3   workspaces   11.0     50       22.00
4         food    0.0      1        0.00



### Response rate by location (over total notifications)

In [15]:
total_byloc = pd.DataFrame({'count': merged_notifications.groupby('locationCommonName')['objectId'].count()}).reset_index()
valid_byloc = pd.DataFrame({'count': valid_notifications.groupby('locationCommonName')['objectId'].count()}).reset_index()
idk_byloc = pd.DataFrame({'count': idk_notifications.groupby('locationCommonName')['objectId'].count()}).reset_index()
non_missed_byloc = pd.DataFrame({'count': non_missed_notifications.groupby('locationCommonName')['objectId'].count()}).reset_index()
missed_byloc = pd.DataFrame({'count': missed_notifications.groupby('locationCommonName')['objectId'].count()}).reset_index()

#### Over Total Notifications

In [16]:
total_valid_byloc = valid_byloc.merge(total_byloc, on='locationCommonName', how='outer')
total_valid_byloc.fillna(0, inplace=True)
total_valid_byloc.rename(columns={'count_x': 'count', 'count_y': 'total'}, inplace=True)
total_valid_byloc['proportion'] = round(100 * total_valid_byloc['count'] / total_valid_byloc['total'], 2)

total_idk_byloc = idk_byloc.merge(total_byloc, on='locationCommonName', how='outer')
total_idk_byloc.fillna(0, inplace=True)
total_idk_byloc.rename(columns={'count_x': 'count', 'count_y': 'total'}, inplace=True)
total_idk_byloc['proportion'] = round(100 * total_idk_byloc['count'] / total_idk_byloc['total'], 2)

total_missed_byloc = missed_byloc.merge(total_byloc, on='locationCommonName', how='outer')
total_missed_byloc.fillna(0, inplace=True)
total_missed_byloc.rename(columns={'count_x': 'count', 'count_y': 'total'}, inplace=True)
total_missed_byloc['proportion'] = round(100 * total_missed_byloc['count'] / total_missed_byloc['total'], 2)

In [124]:
print('Valid Responses by Location: \n{}\n'.format(total_valid_byloc))
print('IDK Responses by Location: \n{}\n'.format(total_idk_byloc))
print('Missed Notifications by Location: \n{}\n'.format(total_missed_byloc))

Valid Responses by Location: 
             locationCommonName  count  total  proportion
0                                  1.0      1      100.00
1                     Blomquist    5.0     26       19.23
2        Brew Bike in Annenberg    7.0      7      100.00
3                    Coffee Lab   20.0     60       33.33
4         Delta Lab/Hackerspace   14.0     25       56.00
5   Food for Thought at Garrett    7.0     57       12.28
6        Kregse Hall Entry Area   24.0     68       35.29
7                    Lisas Cafe    8.0     25       32.00
8             Main Library Core    1.0      2       50.00
9                      Norbucks    1.0     10       10.00
10                Norris Dunkin    4.0      6       66.67
11                 Peets Coffee    4.0     25       16.00
12                         SPAC    4.0     13       30.77
13             Starbucks Coffee    9.0     55       16.36
14                 Tech Express    1.0      5       20.00
15                       Patten    0.0    

#### Over Non-Missed Notifications

In [18]:
non_missed_valid_byloc = valid_byloc.merge(non_missed_byloc, on='locationCommonName', how='outer')
non_missed_valid_byloc.fillna(0, inplace=True)
non_missed_valid_byloc.rename(columns={'count_x': 'count', 'count_y': 'total'}, inplace=True)
non_missed_valid_byloc['proportion'] = round(100 * non_missed_valid_byloc['count'] / non_missed_valid_byloc['total'], 2)

non_missed_idk_byloc = idk_byloc.merge(non_missed_byloc, on='locationCommonName', how='outer')
non_missed_idk_byloc.fillna(0, inplace=True)
non_missed_idk_byloc.rename(columns={'count_x': 'count', 'count_y': 'total'}, inplace=True)
non_missed_idk_byloc['proportion'] = round(100 * non_missed_idk_byloc['count'] / non_missed_idk_byloc['total'], 2)

In [19]:
print('Valid Responses by Location: \n{}\n'.format(non_missed_valid_byloc))
print('IDK Responses by Location: \n{}\n'.format(non_missed_idk_byloc))

Valid Responses by Location: 
             locationCommonName  count  total  proportion
0                                  1.0      1      100.00
1                     Blomquist    5.0     10       50.00
2        Brew Bike in Annenberg    7.0      7      100.00
3                    Coffee Lab   20.0     38       52.63
4         Delta Lab/Hackerspace   14.0     14      100.00
5   Food for Thought at Garrett    7.0     23       30.43
6        Kregse Hall Entry Area   24.0     35       68.57
7                    Lisas Cafe    8.0     10       80.00
8             Main Library Core    1.0      1      100.00
9                      Norbucks    1.0      1      100.00
10                Norris Dunkin    4.0      4      100.00
11                 Peets Coffee    4.0      7       57.14
12                         SPAC    4.0      7       57.14
13             Starbucks Coffee    9.0     23       39.13
14                 Tech Express    1.0      1      100.00
15                       Patten    0.0    

## Analyze how much data has scaffolded for each location

In [125]:
# In general
number_scaffolded_responses = pd.DataFrame({'count': valid_notifications.groupby('hotspotId')['objectId'].count()}).reset_index()
print('Number Unique Hotspots: {}\n'.format(len(number_scaffolded_responses)))
print('Descriptive Statistics of Scaffolded Responses: \n {}'.format(number_scaffolded_responses.describe(percentiles=np.arange(0, 1, 0.05))))

Number Unique Hotspots: 83

Descriptive Statistics of Scaffolded Responses: 
            count
count  83.000000
mean    1.325301
std     0.645971
min     1.000000
0%      1.000000
5%      1.000000
10%     1.000000
15.0%   1.000000
20%     1.000000
25%     1.000000
30.0%   1.000000
35%     1.000000
40%     1.000000
45%     1.000000
50%     1.000000
55.0%   1.000000
60.0%   1.000000
65%     1.000000
70%     1.000000
75%     1.500000
80%     2.000000
85.0%   2.000000
90%     2.000000
95%     2.000000
max     4.000000


## Time between receiving notification and responding

In [44]:
non_missed_notifications = merged_notifications[~merged_notifications['response'].isin(['missed notification'])].reset_index()
non_missed_notifications['time_between'] = non_missed_notifications['timestamp_response'] - non_missed_notifications['timestamp_sent']

In [45]:
non_missed_notifications

,index,createdAt,gmtOffset,hotspotId,notificationString,objectId,timestamp_sent,updatedAt,vendorId,timestamp_response,question,response,tag,locationCommonName,vendor_hotspot_id,time_between
0,2,1494266124,-18000,e3j2GqkyMA,"Notified for e3j2GqkyMA (42.056922, -87.676545...",zyHdCC8auP,1494266124,1494266124,FDFE420C-04ED-47D6-AADF-8909DD559659,1494266137,tablesavailable,yes,workspaces,Delta Lab/Hackerspace,FDFE420C-04ED-47D6-AADF-8909DD559659_e3j2GqkyMA,13
1,5,1494302551,-18000,8xoMpll7UZ,"Notified for 8xoMpll7UZ (42.060275, -87.675714...",Dm1dUaAF4M,1494302488,1494302551,83FF9726-945C-4D9B-9475-A41DC2A41CF4,1494302551,linelength,short: < 5 people,cafes,Lisas Cafe,83FF9726-945C-4D9B-9475-A41DC2A41CF4_8xoMpll7UZ,63
2,42,1494354230,-18000,8tyanI2cDs,Notified for beacon region RsX6mIV5p0,SSoq7oOSLR,1494354229,1494354230,082918FC-40D0-4E15-916F-B0DC4DCE8A7A,1494354371,linelength,short: < 5 people,cafes,Brew Bike in Annenberg,082918FC-40D0-4E15-916F-B0DC4DCE8A7A_8tyanI2cDs,142
3,44,1494361244,-18000,GKyoBvb2VT,"Notified for GKyoBvb2VT (42.055781, -87.674974...",AHYoCdA20I,1494361224,1494361244,082918FC-40D0-4E15-916F-B0DC4DCE8A7A,1494361230,linelength,I don't know,cafes,Food for Thought at Garrett,082918FC-40D0-4E15-916F-B0DC4DCE8A7A_GKyoBvb2VT,6
4,45,1494363736,-18000,5t5BetXvyS,"Notified for 5t5BetXvyS (42.054274, -87.678216...",ZOTQOrZKjI,1494363736,1494363736,F2F4A0A8-98C7-474C-91DF-B6CA1A22B59B,1494363747,cardio,I don't know,gyms,Blomquist,F2F4A0A8-98C7-474C-91DF-B6CA1A22B59B_5t5BetXvyS,11
5,46,1494366652,-18000,HBzYGSKU51,Notified for beacon region CYtXsryXtJ,OIzxRFqmjH,1494366652,1494366652,082918FC-40D0-4E15-916F-B0DC4DCE8A7A,1494366656,tablesavailable,yes,workspaces,Delta Lab/Hackerspace,082918FC-40D0-4E15-916F-B0DC4DCE8A7A_HBzYGSKU51,4
6,49,1494368915,-18000,HBzYGSKU51,Notified for beacon region CYtXsryXtJ,QlEPDg9Fvb,1494368914,1494368915,F2F4A0A8-98C7-474C-91DF-B6CA1A22B59B,1494368997,tabletype,communal tables,workspaces,Delta Lab/Hackerspace,F2F4A0A8-98C7-474C-91DF-B6CA1A22B59B_HBzYGSKU51,83
7,53,1494381637,-18000,2LWxaFQwdh,"Notified for 2LWxaFQwdh (42.051578, -87.675496...",adTxjsobTF,1494381631,1494381637,00BF36C5-3E54-4BA3-8886-8D94942F6053,1494381659,tablesavailable,yes,workspaces,Kregse Hall Entry Area,00BF36C5-3E54-4BA3-8886-8D94942F6053_2LWxaFQwdh,28
8,55,1494382331,-18000,fD065ugN13,"Notified for fD065ugN13 (42.049716, -87.681993...",Rmeq5Knm9K,1494382331,1494382331,00BF36C5-3E54-4BA3-8886-8D94942F6053,1494382560,smalltables,I don't know,coffeeshops,Starbucks Coffee,00BF36C5-3E54-4BA3-8886-8D94942F6053_fD065ugN13,229
9,61,1494425430,-18000,XQ2UFAG45J,Notified for beacon region CYtXsryXtJ,fHt2PvfImE,1494425430,1494425430,FDFE420C-04ED-47D6-AADF-8909DD559659,1494426145,tablesavailable,yes,workspaces,Delta Lab/Hackerspace,FDFE420C-04ED-47D6-AADF-8909DD559659_XQ2UFAG45J,715


### Summary stats for time between (all response types, in seconds)

In [110]:
non_missed_notifications['time_between'].describe(percentiles=np.arange(0, 1, 0.05))

count      183.000000
mean       725.765027
std       2185.198282
min          3.000000
0%           3.000000
5%           5.000000
10%          6.000000
15.0%        8.000000
20%         11.000000
25%         14.500000
30.0%       22.000000
35%         27.700000
40%         41.000000
45%         52.900000
50%         64.000000
55.0%       94.100000
60.0%      149.400000
65%        190.000000
70%        266.400000
75%        328.000000
80%        424.600000
85.0%      704.400000
90%       1677.400000
95%       3981.000000
max      16662.000000
Name: time_between, dtype: float64

### Summary stats for time between (not including idk responses, in seconds)

In [111]:
non_idk_timebetween = non_missed_notifications[~non_missed_notifications['response'].isin(['I don\'t know'])]['time_between']
non_idk_timebetween.describe(percentiles=np.arange(0, 1, 0.05))

count      110.000000
mean       560.200000
std       1654.997163
min          3.000000
0%           3.000000
5%           5.000000
10%          7.900000
15.0%       11.000000
20%         12.800000
25%         18.000000
30.0%       24.000000
35%         27.150000
40%         37.800000
45%         47.150000
50%         60.000000
55.0%       70.950000
60.0%       94.400000
65%        147.950000
70%        191.800000
75%        270.000000
80%        370.800000
85.0%      576.500000
90%       1318.600000
95%       3446.250000
max      13585.000000
Name: time_between, dtype: float64

### Summary stats for time between (only idk responses, in seconds)

In [112]:
idk_timebetween = non_missed_notifications[non_missed_notifications['response'].isin(['I don\'t know'])]['time_between']
idk_timebetween.describe(percentiles=np.arange(0, 1, 0.05))

count       73.000000
mean       975.246575
std       2796.215363
min          4.000000
0%           4.000000
5%           5.600000
10%          6.000000
15.0%        6.000000
20%          7.400000
25%         11.000000
30.0%       15.400000
35%         28.600000
40%         46.600000
45%         61.000000
50%        140.000000
55.0%      163.800000
60.0%      202.600000
65%        276.800000
70%        296.800000
75%        341.000000
80%        540.400000
85.0%      970.800000
90%       2300.000000
95%       4359.800000
max      16662.000000
Name: time_between, dtype: float64

### Summary stats parsed out by location category (all responses, in seconds)

In [116]:
non_missed_notifications.groupby('tag')['time_between'].describe(percentiles=np.arange(0, 1, 0.05)).transpose()

tag,cafes,coffeeshops,food,gyms,workspaces
count,41.000000,73.000000,1.0,18.000000,50.00000
mean,157.487805,1198.150685,429.0,288.722222,665.34000
std,389.838436,3135.014295,NaN,655.578936,1530.66385
min,3.000000,5.000000,429.0,5.000000,4.00000
0%,3.000000,5.000000,429.0,5.000000,4.00000
5%,5.000000,7.600000,429.0,5.850000,4.45000
10%,5.000000,9.200000,429.0,6.000000,6.00000
15.0%,6.000000,16.800000,429.0,6.000000,11.00000
20%,7.000000,22.800000,429.0,6.400000,11.80000
25%,8.000000,26.000000,429.0,7.250000,13.25000


### Summary stats parsed out by location category (not including idk responses, in seconds)

In [121]:
non_missed_notifications[~non_missed_notifications['response'].isin(['I don\'t know'])].groupby('tag')['time_between'].describe(percentiles=np.arange(0, 1, 0.05)).transpose()

tag,cafes,coffeeshops,food,gyms,workspaces
count,23.000000,38.000000,1.0,9.000000,39.000000
mean,193.956522,663.289474,429.0,480.222222,697.564103
std,501.161150,2269.616159,NaN,888.267102,1554.471916
min,3.000000,5.000000,429.0,7.000000,4.000000
0%,3.000000,5.000000,429.0,7.000000,4.000000
5%,5.000000,7.850000,429.0,7.400000,4.900000
10%,5.200000,8.700000,429.0,7.800000,10.200000
15.0%,7.500000,20.200000,429.0,8.800000,11.000000
20%,12.600000,22.800000,429.0,10.400000,12.000000
25%,16.500000,24.000000,429.0,12.000000,13.500000


### Summary stats parsed out by location category (only idk responses, in seconds)

In [122]:
non_missed_notifications[non_missed_notifications['response'].isin(['I don\'t know'])].groupby('tag')['time_between'].describe(percentiles=np.arange(0, 1, 0.05)).transpose()

tag,cafes,coffeeshops,gyms,workspaces
count,18.000000,35.000000,9.000000,11.000000
mean,110.888889,1778.857143,97.222222,551.090909
std,168.681701,3813.182314,204.351277,1509.868501
min,4.000000,6.000000,5.000000,4.000000
0%,4.000000,6.000000,5.000000,4.000000
5%,4.850000,7.700000,5.400000,5.000000
10%,5.700000,10.400000,5.800000,6.000000
15.0%,6.000000,12.800000,6.000000,6.000000
20%,6.400000,36.800000,6.000000,6.000000
25%,7.000000,96.000000,6.000000,15.500000


## Contributions by Time of Day